In [ ]:
!pip install transformers==4.30.0 accelerate==0.21.0 datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 48.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 22.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 41.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 91.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 18.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 27.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 21.0 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)


In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
from datasets import load_dataset

# mlsum datasetini yükle
data_size = 4
dataset = load_dataset("mlsum", "tu", split=f'train[:{data_size}%]')

# Train ve validation setleri
train_test_split = dataset.train_test_split(test_size=0.2)
train_dataset = train_test_split['train']
val_dataset = train_test_split['test']

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


The repository for mlsum contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/mlsum.
You can avoid this prompt in future by passing the argument `trust_remote_code=True`.

Do you wish to run the custom code? [y/N] y


Generating train split:   0%|          | 0/249277 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/11565 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/12775 [00:00<?, ? examples/s]

In [ ]:
from transformers import BartForConditionalGeneration, BartTokenizer

model_name = "facebook/bart-large-cnn"
model = BartForConditionalGeneration.from_pretrained(model_name)
tokenizer = BartTokenizer.from_pretrained(model_name)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
def preprocess_data(examples):
    inputs = [doc for doc in examples['text']]
    model_inputs = tokenizer(inputs, max_length=512, truncation=True, padding="max_length", return_tensors='pt')

    # Özeti hedef olarak belirleyelim
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(examples['summary'], max_length=150, truncation=True, padding="max_length", return_tensors='pt')

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs


# Veri setini işlemden geçir
train_dataset = train_dataset.map(preprocess_data, batched=True, remove_columns=["text", "summary"])
val_dataset = val_dataset.map(preprocess_data, batched=True, remove_columns=["text", "summary"])

Map:   0%|          | 0/7976 [00:00<?, ? examples/s]

/usr/local/lib/python3.10/dist-packages/transformers/tokenization_utils_base.py:3619: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(


Map:   0%|          | 0/1995 [00:00<?, ? examples/s]

In [ ]:
from torch.utils.data import DataLoader

train_dataloader = DataLoader(train_dataset, batch_size=8, shuffle=True)
val_dataloader = DataLoader(val_dataset, batch_size=8)


In [ ]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./results",
    evaluation_strategy="epoch",
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    num_train_epochs=3,
    weight_decay=0.01,
    save_total_limit=3,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
)

trainer.train()


/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch,Training Loss,Validation Loss
1,0.908600,0.663191
2,0.600000,0.617418
3,0.510400,0.603648


TrainOutput(global_step=2991, training_loss=0.637822928216616, metrics={'train_runtime': 1671.3108, 'train_samples_per_second': 14.317, 'train_steps_per_second': 1.79, 'total_flos': 2.592723945966797e+16, 'train_loss': 0.637822928216616, 'epoch': 3.0})

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from os import path
trainer.evaluate()
path = '/content/drive/MyDrive/nlp/_10000'
model.save_pretrained(path)
tokenizer.save_pretrained(path)


('/content/drive/MyDrive/nlp/_10000/tokenizer_config.json',
 '/content/drive/MyDrive/nlp/_10000/special_tokens_map.json',
 '/content/drive/MyDrive/nlp/_10000/vocab.json',
 '/content/drive/MyDrive/nlp/_10000/merges.txt',
 '/content/drive/MyDrive/nlp/_10000/added_tokens.json')

In [ ]:
import torch
from transformers import BartForConditionalGeneration, BartTokenizer

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

path = '/content/drive/MyDrive/nlp/_2500'
model = BartForConditionalGeneration.from_pretrained(path).to(device)
tokenizer = BartTokenizer.from_pretrained(path)

In [ ]:
path2 = '/content/drive/MyDrive/nlp/_10000'
model2 = BartForConditionalGeneration.from_pretrained(path2).to(device)
tokenizer2 = BartTokenizer.from_pretrained(path2)

In [ ]:
test_text = """
ABD’DEKİ Harvard Üniversitesi tarafından yapılan bir araştırmada, “agresif, çabuk sinirlenen, hiç kimsenin ne düşündüğünü önemsemeyen ve hoşgörüsüz” insanların, evrim basamağında, sakin ve yardımsever insanlara göre daha yüksekte yer aldığı öne sürüldü. Harvard’lı bilim adamlarına göre, yardımseverlik veya paylaşma duyguları, “evrim olarak daha az gelişmiş insanlara” has duygular. Uzmanlar bu çalışma için biyolojik, sosyallik ve davranış olarak daha medeni veya ileri olan şempanzeler ile evrimin daha alt basamaklarında yer alan bir diğer maymun türü olan bonoboları karşılaştırdı. Sonuçta şempanzelerin, daha agresif, sinirli, ve bencil olduğu ortaya çıktı. Araştırmanın başındaki Victoria Wobber’a göre, paylaşmak ve iyi niyet, “evrimsel” olarak, bağların sıkılaştırılması dönemine denk geliyor. Bağlar ve sosyal yaşam sıkılaştıktan sonra da bir sonraki aşama, agresif ve bencillik oluyor.
"""

# Metni tokenize etme ve özetleme
inputs = tokenizer(test_text, max_length=1024, truncation=True, return_tensors='pt').to(device)
summary_ids = model.generate(inputs['input_ids'], max_length=150, num_beams=4, length_penalty=2.0, early_stopping=True)

# Özetin decode edilmesi
summary = tokenizer.decode(summary_ids[0], skip_special_tokens=True)
print("Özet:", summary)

Özet: ABD'DEKİ Harvard Üniversitesi tarafından yapılan bir araştırmada, evrim basamağında sakin ve yardımsever insanlara göre daha yüksekte yer aldığı öne sürüldü.


In [ ]:
inputs2 = tokenizer2(test_text, max_length=1024, truncation=True, return_tensors='pt').to(device)
summary_ids2 = model2.generate(inputs2['input_ids'], max_length=150, num_beams=4, length_penalty=2.0, early_stopping=True)

# Özetin decode edilmesi
summary2 = tokenizer2.decode(summary_ids2[0], skip_special_tokens=True)
print("Özet:", summary2)

Özet: Harvard Üniversitesi tarafından yapılan bir araştırmada, evrim basamağında, sakin ve yardımsever insanlara göre daha yüksekte yer aldığı öne sürüldü.


In [ ]:
print(dataset[66]['text'])
print(dataset[66]['summary'])

ABD’DEKİ Harvard Üniversitesi tarafından yapılan bir araştırmada, “agresif, çabuk sinirlenen, hiç kimsenin ne düşündüğünü önemsemeyen ve hoşgörüsüz” insanların, evrim basamağında, sakin ve yardımsever insanlara göre daha yüksekte yer aldığı öne sürüldü. Harvard’lı bilim adamlarına göre, yardımseverlik veya paylaşma duyguları, “evrim olarak daha az gelişmiş insanlara” has duygular. Uzmanlar bu çalışma için biyolojik, sosyallik ve davranış olarak daha medeni veya ileri olan şempanzeler ile evrimin daha alt basamaklarında yer alan bir diğer maymun türü olan bonoboları karşılaştırdı. Sonuçta şempanzelerin, daha agresif, sinirli, ve bencil olduğu ortaya çıktı. Araştırmanın başındaki Victoria Wobber’a göre, paylaşmak ve iyi niyet, “evrimsel” olarak, bağların sıkılaştırılması dönemine denk geliyor. Bağlar ve sosyal yaşam sıkılaştıktan sonra da bir sonraki aşama, agresif ve bencillik oluyor.
Hardvard Üniversitesi araştırdı. Ortaya çıkan sonuç ilginç. Meğer Recep İvedik'in bazı huyları evrimin 